# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process, LLM, Agent

# Create a manager agent for hierarchical process (must have no tools)
manager_agent = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex "
              "projects and guiding teams to success. Your role is to coordinate the "
              "efforts of the crew members, ensuring that each task is completed on time "
              "and to the highest standard.",
    allow_delegation=True,
    verbose=True
)

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_agent=manager_agent,
    # manager_llm=LLM(model="gpt-4o-mini", temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a0676789-ff78-45f1-a7d1-6a4565460531                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Manager                                                                                         │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling     │
│  and machine learning to identify trends and predict market movements.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Manager                                                                                         │
│                                                                                                                 │
│  Thought: Thought: I need to gather market data for AAPL and identify insights that could lead to trends and    │
│  predictions regarding its movements. I will start by searching for relevant market data and analysis on AAPL.  │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  In evaluating the risks associated with proposed trading strategies for AAPL as of October 2023, several key   │
│  insights arise from recent financial performance, market trends, and competitive dynamics.                     │
│                                                                                                                 │
│  1. **Volatility and Recent Performance**: AAPL experienced a substantial volatility drop of 11.6% during Q3    │
│  2023, routed mainly from a broader market sell-off, despite earlier gains in the year. The forecasted share    │
│  price around $157.54 suggests potential short-term gains, but underlying volatility presents a challenge.      │
│                                                                                                                 │
│  2. **Valuation Perspective**: Analysts from Morningstar estimate AAPL's fair value at $150, indicating that    │
│  the stock may currently be overvalued. Without innovative product releases or surge in demand, AAPL might not  │
│  see substantial price growth, with year-end 2025 targets modestly predicting a range of $220-$235.             │
│                                                                                                                 │
│  3. **Competitive Landscape and Growth Concerns**: The competitive atmosphere, particularly from companies      │
│  like Huawei, raises concerns about iPhone demand which is projected to grow at only a 3% CAGR. The growing     │
│  emphasis on services and wearable technology provides some solace; however, potential weakness in overall      │
│  sales growth adds pressure.                                                                                    │
│                                                                                                                 │
│  4. **Earnings Report Insights**: The Q4 earnings report indicated a slight revenue decrease of 1%, but         │
│  services revenue hit a record high. This mixed performance shows the company’s strengths in diversification    │
│  but doesn’t alleviate risks associated with declining core product sales, such as iPhones.                     │
│                                                                                                                 │
│  5. **Market Sentiment**: The general sentiment around the stock market is cautiously optimistic; however,      │
│  economic factors including supply chain vulnerabilities and external market conditions will impact AAPL's      │
│  stock performance.                                                                                             │
│                                                                                                                 │
│  6. **Mitigation Strategies**:                                                                                  │
│     - **Diversified Approaches**: Implementing a combination of day trading and longer-term investment          │
│  strategies may provide flexibility against volatility. Traders can use historical backtesting strategies to    │
│  optimize entry points and risk m...                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Manager                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  In evaluating the risks associated with proposed trading strategies for AAPL as of October 2023, several key   │
│  insights arise from recent financial performance, market trends, and competitive dynamics.                     │
│                                                                                                                 │
│  1. **Volatility and Recent Performance**: AAPL experienced a substantial volatility drop of 11.6% during Q3    │
│  2023, routed mainly from a broader market sell-off, despite earlier gains in the year. The forecasted share    │
│  price around $157.54 suggests potential short-term gains, but underlying volatility presents a challenge.      │
│                                                                                                                 │
│  2. **Valuation Perspective**: Analysts from Morningstar estimate AAPL's fair value at $150, indicating that    │
│  the stock may currently be overvalued. Without innovative product releases or surge in demand, AAPL might not  │
│  see substantial price growth, with year-end 2025 targets modestly predicting a range of $220-$235.             │
│                                                                                                                 │
│  3. **Competitive Landscape and Growth Concerns**: The competitive atmosphere, particularly from companies      │
│  like Huawei, raises concerns about iPhone demand which is projected to grow at only a 3% CAGR. The growing     │
│  emphasis on services and wearable technology provides some solace; however, potential weakness in overall      │
│  sales growth adds pressure.                                                                                    │
│                                                                                                                 │
│  4. **Earnings Report Insights**: The Q4 earnings report indicated a slight revenue decrease of 1%, but         │
│  services revenue hit a record high. This mixed performance shows the company’s strengths in diversification    │
│  but doesn’t alleviate risks associated with declining core product sales, such as iPhones.                     │
│                                                                                                                 │
│  5. **Market Sentiment**: The general sentiment around the stock market is cautiously optimistic; however,      │
│  economic factors including supply chain vulnerabilities and external market conditions will impact AAPL's      │
│  stock performance.                                                                                             │
│                                                                                                                 │
│  6. **Mitigation Strategies**:                                                                                  │
│     - **Diversified Approaches**: Implementing a combination of day trading and longer-term investment          │
│  strategies may provide flexibility against volatility. Traders can use historical backtesting strategies to    │
│  optimize entry points and risk management.                                                                     │
│     - **Active Monitoring of Market Conditions**: Continuous awareness of market conditions, quarterly          │
│  earnings reports, and shifts in competitive dynamics i

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 29304922-7b55-4a48-9a74-7a27cf55a58f                                                                     │
│  Agent: Project Manager                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a0676789-ff78-45f1-a7d1-6a4565460531                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: In evaluating the risks associated with proposed trading strategies for AAPL as of October       │
│  2023, several key insights arise from recent financial performance, market trends, and competitive dynamics.   │
│                                                                                                                 │
│  1. **Volatility and Recent Performance**: AAPL experienced a substantial volatility drop of 11.6% during Q3    │
│  2023, routed mainly from a broader market sell-off, despite earlier gains in the year. The forecasted share    │
│  price around $157.54 suggests potential short-term gains, but underlying volatility presents a challenge.      │
│                                                                                                                 │
│  2. **Valuation Perspective**: Analysts from Morningstar estimate AAPL's fair value at $150, indicating that    │
│  the stock may currently be overvalued. Without innovative product releases or surge in demand, AAPL might not  │
│  see substantial price growth, with year-end 2025 targets modestly predicting a range of $220-$235.             │
│                                                                                                                 │
│  3. **Competitive Landscape and Growth Concerns**: The competitive atmosphere, particularly from companies      │
│  like Huawei, raises concerns about iPhone demand which is projected to grow at only a 3% CAGR. The growing     │
│  emphasis on services and wearable technology provides some solace; however, potential weakness in overall      │
│  sales growth adds pressure.                                                                                    │
│                                                                                                                 │
│  4. **Earnings Report Insights**: The Q4 earnings report indicated a slight revenue decrease of 1%, but         │
│  services revenue hit a record high. This mixed performance shows the company’s strengths in diversification    │
│  but doesn’t alleviate risks associated with declining core product sales, such as iPhones.                     │
│                                                                                                                 │
│  5. **Market Sentiment**: The general sentiment around the stock market is cautiously optimistic; however,      │
│  economic factors including supply chain vulnerabilities and external market conditions will impact AAPL's      │
│  stock performance.                                                                                             │
│                                                                                                                 │
│  6. **Mitigation Strategies**:                                                                                  │
│     - **Diversified Approaches**: Implementing a combination of day trading and longer-term investment          │
│  strategies may provide flexibility against volatility. Traders can use historical backtesting strategies to    │
│  optimize entry points and risk management.                                                                     │
│     - **Active Monitoring of Market Conditions**: Cont

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result.raw)

In evaluating the risks associated with proposed trading strategies for AAPL as of October 2023, several key insights arise from recent financial performance, market trends, and competitive dynamics.

1. **Volatility and Recent Performance**: AAPL experienced a substantial volatility drop of 11.6% during Q3 2023, routed mainly from a broader market sell-off, despite earlier gains in the year. The forecasted share price around $157.54 suggests potential short-term gains, but underlying volatility presents a challenge.

2. **Valuation Perspective**: Analysts from Morningstar estimate AAPL's fair value at $150, indicating that the stock may currently be overvalued. Without innovative product releases or surge in demand, AAPL might not see substantial price growth, with year-end 2025 targets modestly predicting a range of $220-$235.

3. **Competitive Landscape and Growth Concerns**: The competitive atmosphere, particularly from companies like Huawei, raises concerns about iPhone demand which is projected to grow at only a 3% CAGR. The growing emphasis on services and wearable technology provides some solace; however, potential weakness in overall sales growth adds pressure.

4. **Earnings Report Insights**: The Q4 earnings report indicated a slight revenue decrease of 1%, but services revenue hit a record high. This mixed performance shows the company’s strengths in diversification but doesn’t alleviate risks associated with declining core product sales, such as iPhones.

5. **Market Sentiment**: The general sentiment around the stock market is cautiously optimistic; however, economic factors including supply chain vulnerabilities and external market conditions will impact AAPL's stock performance.

6. **Mitigation Strategies**:
   - **Diversified Approaches**: Implementing a combination of day trading and longer-term investment strategies may provide flexibility against volatility. Traders can use historical backtesting strategies to optimize entry points and risk management.
   - **Active Monitoring of Market Conditions**: Continuous awareness of market conditions, quarterly earnings reports, and shifts in competitive dynamics is critical. Leveraging analytical tools to monitor stock movements can provide traders with timely data to make informed decisions.
   - **Risk Adjusted Position Sizing**: Adjusting position sizes based on volatility forecasts and market sentiment will help in managing adverse movements in stock value. Setting stop-loss limits can offer additional protection.

In conclusion, while AAPL remains a strong long-term investment with its substantial market position and revenue capabilities from new segments, the combination of valuation pressures, market volatility, and competitive threats necessitates caution in proposed trading activities. Active risk management and strategic adjustments will be essential to navigate this dynamic environment effectively.